In [1]:
# Web_Scrape_SQL_Analysis.ipynb

# 🧱 Step 1: Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# 🔐 Step 2: Load environment variables
load_dotenv()
pg_user = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")
pg_host = os.getenv("PG_HOST")
pg_db = os.getenv("PG_DB")

# 📊 Step 3: Connect to PostgreSQL engine
engine = create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}/{pg_db}')

# 📃 Set pandas to display all rows
pd.set_option('display.max_rows', None)


In [3]:
# 🧠 Business Question:
# Which players are nearing the end of their contracts, and how many minutes have they played this season?
# Sort by contribution to easily assess who to retain or release.

# 📾 SQL Query
query_web_contracts = '''
SELECT 
    name,
    position,
    contract_end,
    REGEXP_REPLACE(minutes_played, '[^0-9]', '', 'g')::INTEGER AS minutes_played
FROM raw.raw_web_players
WHERE contract_end <= '2026-06'
ORDER BY minutes_played DESC;
'''

# 🔪 Execute the SQL Query
contract_expiry_df = pd.read_sql(query_web_contracts, engine)
contract_expiry_df

,name,position,contract_end,minutes_played
0,Bernardo Silva,Midfield,2026-06,2400
1,Ilkay Gündogan,Midfield,2026-06,2052
2,Ederson,Goalkeeper,2026-06,2051
3,Kevin De Bruyne,Midfield,2025-06,1542
4,Stefan Ortega Moreno,Goalkeeper,2026-06,1099
5,John Stones,Defence,2026-06,547
6,Nico O'Reilly,Midfield,2025-06,513
7,James McAtee,Midfield,2026-06,294
8,Jahmai Simpson-Pusey,Defence,2026-06,95
9,Divin Mubama,Offence,2025-06,27


In [ ]:
#📊 Insight
#Among the players whose contracts end by June 2026, a stark contrast is observed in match 
# contribution. Bernardo Silva, İlkay Gündoğan, and Ederson are among the top contributors 
# with over 2000 minutes played. In contrast, 7 players have recorded zero minutes, 
# highlighting a gap in utilization.

#💡 Recommendation
#Prioritize contract renewals for players who have made substantial contributions, 
# particularly those with over 1500 minutes played. For players with zero or negligible
#  minutes, assess injury status, training performance, or loan/transfer opportunities to 
# optimize squad value and reduce wage inefficiencies.

#🔮 Prediction
#If current utilization trends continue, unutilized players are unlikely to see increased 
# playing time in the remaining season. Expect increased likelihood of squad turnover in 
# the next transfer window unless there are significant changes in rotation or injury 
# recovery.




In [2]:
# 🧱 Step 1: Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# 🔐 Step 2: Load environment variables
load_dotenv()
pg_user = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")
pg_host = os.getenv("PG_HOST")
pg_db = os.getenv("PG_DB")

# 📂 Step 3: Connect to PostgreSQL
engine = create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}/{pg_db}')

# 📋 Set pandas to display all rows
pd.set_option('display.max_rows', None)

In [3]:
# 🧠 Business Question:
# Are there contract patterns or position-based trends among players who contributed the least in minutes this season?
# Use CTE to isolate bottom 25% in contribution and compare each to their positional average.

# 🗓 SQL Query
query_diagnostic_web = '''
WITH cleaned_players AS (
    SELECT 
        name,
        position,
        contract_end,
        REGEXP_REPLACE(minutes_played, '[^0-9]', '', 'g')::INTEGER AS minutes_played
    FROM raw.raw_web_players
),
ranked_players AS (
    SELECT *,
           PERCENT_RANK() OVER (ORDER BY minutes_played) AS pct_rank,
           AVG(minutes_played) OVER (PARTITION BY position) AS avg_position_minutes
    FROM cleaned_players
)
SELECT 
    name,
    position,
    contract_end,
    minutes_played,
    avg_position_minutes,
    ROUND(minutes_played - avg_position_minutes, 2) AS difference_from_avg
FROM ranked_players
WHERE pct_rank <= 0.25
ORDER BY minutes_played ASC
LIMIT 10;
'''

# 🔪 Execute the SQL Query
diagnostic_df = pd.read_sql(query_diagnostic_web, engine)
diagnostic_df



,name,position,contract_end,minutes_played,avg_position_minutes,difference_from_avg
0,Max Alleyne,Defence,2025-06,0,866.750000,-866.75
1,Vitor Reis,Defence,2029-06,0,866.750000,-866.75
2,Christian McFarlane,Defence,2025-06,0,866.750000,-866.75
3,Farid Alfa-Ruprecht,Offence,2026-06,0,878.750000,-878.75
4,Claudio Echeverri,Midfield,2028-06,0,1107.933333,-1107.93
5,Spike Brits,Goalkeeper,2025-06,0,630.000000,-630.00
6,Scott Carson,Goalkeeper,2025-06,0,630.000000,-630.00
7,Max Hudson,Goalkeeper,2025-06,0,630.000000,-630.00
8,Kaden Braithwaite,Defence,2025-06,0,866.750000,-866.75


In [ ]:
#🔍 Insight
#The bottom 25% of players in terms of minutes played all recorded 0 minutes, with the 
# exception of a few who contributed marginally (e.g., 27 or 95 minutes). Each of these 
# players fell hundreds of minutes below their position's average, with the most severe gap 
# observed in midfielders like Claudio Echeverri who underperformed by over 1,100 minutes
#  compared to the positional norm.

#✅ Recommendation
#The club should consider reevaluating the role and necessity of these players in the 
# current squad. Those with 0 minutes played and contracts ending soon (e.g., Max Alleye,
#  Spike Brits, Scott Carson) are strong candidates for release, loan, or development 
# plans depending on age and potential. Resources could be better allocated toward 
# high-performing or more actively rotated players.

#📈 Prediction
#Unless there is a strategic reason (e.g. youth development or injury recovery), it is 
# likely that these underutilized players will continue to receive minimal playtime 
# next season. Without a change in rotation strategy, they will not significantly impact
#  team performance or value—further justifying contract renegotiation or transfer listing.